Please note that:
- you should use a GPU to run the foollowing code due to high training and testing time
- using cloud computing is recommended
- path relatives to saving the model are to be changed in the code

# Imports

In [1]:
import tensorflow.keras as keras
import tensorflow as tf
import time

import matplotlib

matplotlib.use('agg')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

!pip3 install pickle5 #&> /dev/null
!pip3 install tensorflow_addons #&> /dev/null
import pickle5 as pickle
import tensorflow_addons as tfa


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.model_selection import GroupShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.neural_network import MLPClassifier

tf.autograph.set_verbosity(0)
tf.config.list_physical_devices('GPU')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 KB 3.3 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp310-cp310-macosx_10_9_universal2.whl size=170545 sha256=09e41339feb3334c5bcfc88b80fb22be9fd14ccc1643cf02fa5f627a9350f9f6
  Stored in directory: /Users/tristanstampfler/Library/Caches/pip/wheels/7d/14/ef/4aab19d27fa8e58772be5c71c16add0426acf9e1f64353235c
Successfully built pickle5
You should consider upgrading via the '/Users/tristanstampfler/Documents/ETH/0.Master_Thesis/4. Python/venv/bin/python -m pip install --upgrade pip' command.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.4/588.4 KB 8.2 MB/s eta 0:00:0000:01
You should consider upgrading via the '/Users/tristanstampfler/Documents/ETH/0.Master_Thesis/4. Python/venv/bin/python -m pip install --upgrade pip' command.


[]

# Read the data

In [96]:
path = 'data/'

with open(path + 'adl_and_fall_data.pickle', 'rb') as handle:
    data_raw = pickle.load(handle) #shape: (examples, time_series, channels)

labels = pd.read_csv(path+'adl_and_fall_labels.csv',index_col=0)

# Preprocess the data

In [97]:
compression_factor = 1

def moving_average(a, n=compression_factor) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

### Compress the timestamps
features = []
for k in range(data_raw.shape[2]):
    data_feat_raw = data_raw[:,:,k]
    data_feat = np.zeros((data_feat_raw.shape[0], data_feat_raw.shape[1]//compression_factor))
    for i in range(len(data_feat_raw)):
        data_feat[i] = moving_average(data_feat_raw[i],n=compression_factor)[::compression_factor]
    data_feat = data_feat.reshape(data_feat.shape[0],data_feat.shape[1],1)
    features.append(data_feat)
    
data = np.concatenate(features, axis=2)

# Classification

#### Define Resnet

In [98]:
### Original source code from https://github.com/hfawaz/dl-4-tsc
class Classifier_RESNET:

    def __init__(self, output_directory, input_shape, nb_classes, verbose=False, build=True, 
                 load_weights=False, batch_size = 64, n_feature_maps = 128, subject = None):
        self.n_feature_maps = n_feature_maps
        self.output_directory = output_directory
        if subject is not None:
            self.subject = subject
        if build == True:
            self.model = self.build_model(input_shape, nb_classes)
            self.verbose = verbose
            self.batch_size = batch_size
            if load_weights == True:
                model_path = self.output_directory + f'best_model_subject_{subject}.hdf5'
                self.model = keras.models.load_model(model_path)
        return

    def build_model(self, input_shape, nb_classes):
        n_feature_maps = self.n_feature_maps
        dropout_rate = 0.5
        kernel_size_1 = 8
        kernel_size_2 = 7
        kernel_size_3 = 5

        input_layer = keras.layers.Input(input_shape)

        # BLOCK 1
        conv_x = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=kernel_size_1, padding='same')(input_layer)
        conv_x = keras.layers.BatchNormalization()(conv_x)
        conv_x = keras.layers.Activation('relu')(conv_x)
        conv_x = keras.layers.Dropout(dropout_rate)(conv_x)

        conv_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=kernel_size_2, padding='same')(conv_x)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y = keras.layers.Activation('relu')(conv_y)
        conv_y = keras.layers.Dropout(dropout_rate)(conv_y)

        conv_z = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=kernel_size_2, padding='same')(conv_y)
        conv_z = keras.layers.BatchNormalization()(conv_z)

        # expand channels for the sum
        shortcut_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=1, padding='same')(input_layer)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

        output_block_1 = keras.layers.add([shortcut_y, conv_z])
        output_block_1 = keras.layers.Activation('relu')(output_block_1)
        output_block_1 = keras.layers.Dropout(dropout_rate, seed=0)(output_block_1)

        # BLOCK 2
        conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=kernel_size_1, padding='same')(output_block_1)
        conv_x = keras.layers.BatchNormalization()(conv_x)
        conv_x = keras.layers.Activation('relu')(conv_x)
        conv_x = keras.layers.Dropout(dropout_rate)(conv_x)

        conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=kernel_size_2, padding='same')(conv_x)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y = keras.layers.Activation('relu')(conv_y)
        conv_y = keras.layers.Dropout(dropout_rate)(conv_y)

        conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=kernel_size_2, padding='same')(conv_y)
        conv_z = keras.layers.BatchNormalization()(conv_z)

        # expand channels for the sum
        shortcut_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=1, padding='same')(output_block_1)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

        output_block_2 = keras.layers.add([shortcut_y, conv_z])
        output_block_2 = keras.layers.Activation('relu')(output_block_2)
        output_block_2 = keras.layers.Dropout(dropout_rate, seed=0)(output_block_2)

        # BLOCK 3
        conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=kernel_size_1, padding='same')(output_block_2) #change here
        conv_x = keras.layers.BatchNormalization()(conv_x)
        conv_x = keras.layers.Activation('relu')(conv_x)
        conv_x = keras.layers.Dropout(dropout_rate)(conv_x)

        conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=kernel_size_2, padding='same')(conv_x)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y = keras.layers.Activation('relu')(conv_y)
        conv_y = keras.layers.Dropout(dropout_rate)(conv_y)

        conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=kernel_size_2, padding='same')(conv_y)
        conv_z = keras.layers.BatchNormalization()(conv_z)

        # No need to expand channels because they are equal
        shortcut_y = keras.layers.BatchNormalization()(output_block_2)

        output_block_3 = keras.layers.add([shortcut_y, conv_z])
        output_block_3 = keras.layers.Activation('relu')(output_block_3)

        # FINAL
        gap_layer = keras.layers.GlobalAveragePooling1D()(output_block_3)

        output_layer = keras.layers.Dense(nb_classes, activation='softmax')(gap_layer)

        model = keras.models.Model(inputs=input_layer, outputs=output_layer)

        model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(),
                      metrics=['accuracy'])

        file_path = self.output_directory + f'best_model_subject_{subject}.hdf5'

        model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='loss',
                                                           save_best_only=True)
        
        reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50, min_lr=0.0001)
        self.callbacks = [reduce_lr, model_checkpoint]

        return model

    def fit(self, x_train, y_train, x_val, y_val, nb_epochs, class_weight=None):
        if not tf.test.is_gpu_available:
            print('error')
            exit()
        # x_val and y_val are only used to monitor the test loss and NOT for training
        batch_size = self.batch_size
        nb_epochs = nb_epochs

        mini_batch_size = int(min(x_train.shape[0] / 10, batch_size))

        hist = self.model.fit(x_train, y_train, batch_size=mini_batch_size, epochs=nb_epochs,
                              verbose=self.verbose, validation_data=(x_val, y_val), callbacks=self.callbacks,
                             class_weight=class_weight)
        keras.backend.clear_session()

        return

    def predict(self, x_test):
        model_path = self.output_directory + f'best_model_subject_{subject}.hdf5'
        model = keras.models.load_model(model_path)
        y_pred = model.predict(x_test)

        return y_pred

## Leave-One-Subject-Out Cross Validation

In [99]:
def label_smoothing(y, alpha=0.1):
    """simple label smoothing"""
    K = y.shape[1]
    return y * (1-alpha) + alpha / K

In [100]:
### Initiliaze log files
network = 'resnet'
subjects = np.arange(1,31)

with open(f'sota/{network}/logs.txt','w') as f:
    f.write('average logs\n')
    f.close()

for subject in subjects:
    with open(f'sota/{network}/logs_subject_{subject}.txt', 'w') as f:
        f.write(f'subject {subject} logs\n')
        f.close()

In [ ]:
multi_class = True
sota = True #dont forget to add subject as args when instantiating resnet
alpha = 0.1
n_feature_maps = 256
nb_epochs = 120
load_weights = False
batch_size = 256
seed = 4
verbose = True


### Define data
subsample = 1
X = data[::subsample]
if multi_class:
    y = labels[::subsample]['(is_fall,label)'].to_numpy()
else:
    y = labels[::subsample]['is_fall'].to_numpy()
y = y.reshape(y.shape[0])

### Leave-One-Subject-Out Cross Validation
labels_subjects = labels[::subsample]['subject'].to_numpy().reshape(-1)
metrics = {}
metrics['accuracy'] = []
metrics['f1_score'] = []
metrics['sensitivity'] = []
metrics['specificity'] = []

for subject in subjects:
    subjects_val = np.array([subject])

    mask = np.isin(labels_subjects,subjects_val)

    X_train, y_train, X_val, y_val = X[~mask], y[~mask], X[mask], y[mask]

    ### Transform the labels from integers to one hot vectors
    enc = OneHotEncoder(categories='auto')
    enc.fit(np.concatenate((y_train, y_val), axis=0).reshape(-1, 1))
    y_train_oh = enc.transform(y_train.reshape(-1, 1)).toarray()
    y_val_oh = enc.transform(y_val.reshape(-1, 1)).toarray()
    y_train_oh = label_smoothing(y_train_oh, alpha)
    y_val_oh = label_smoothing(y_val_oh, alpha)

    ### Create network
    if sota:
        output_directory = f'sota/{network}/'
    else:
        output_directory = f'{network}/'
    input_shape = X_train.shape[1:]
    nb_classes = len(y_val_oh[0])

    np.random.seed(seed)
    tf.random.set_seed(seed)

    ### Train
    if network == 'resnet' and sota:
        clf = Classifier_RESNET(output_directory=output_directory, input_shape=input_shape, nb_classes=nb_classes,\
                          verbose=verbose, batch_size = batch_size, n_feature_maps = n_feature_maps, load_weights=load_weights, subject=subject)
    elif network == 'resnet' and not sota:
        clf = Classifier_RESNET(output_directory=output_directory, input_shape=input_shape, nb_classes=nb_classes,\
                          verbose=verbose, batch_size = 256, n_feature_maps = n_feature_maps, load_weights=False)
        
    clf.fit(X_train, y_train_oh, X_val, y_val_oh, nb_epochs)
    y_pred_probs = clf.predict(X_val)
    y_pred = np.argmax(y_pred_probs, axis=1)

    ### Compute metrics
    if multi_class:
        y_val = np.argmax(y_val_oh,axis=1)
    accuracy = accuracy_score(y_val,y_pred)
    f1 = f1_score(y_val,y_pred, average='weighted')
    metrics['accuracy'].append(accuracy)
    metrics['f1_score'].append(f1)
    
    with open(f'sota/{network}/logs_subject_{subject}.txt', 'a') as f:
        f.write(f'accuracy: {np.round(accuracy,4)}\n')
        f.write(f'f1 score: {np.round(f1,4)}\n')
        f.close()

    if not multi_class:
        tn, fp, fn, tp = confusion_matrix(y_val,y_pred).ravel()
        sensitivity = tp / (tp+fn)
        specificity = tn / (tn+fp)
        metrics['sensitivity'].append(sensitivity)
        metrics['specificity'].append(specificity)

print('\n')
print('accuracy: ' + str(np.round(np.average(metrics['accuracy']),4)))
print('accuracy details: ' + str(metrics['accuracy']))
print('f1_score: ' + str(np.round(np.average(metrics['f1_score']),4)))
print('f1_score details: ' + str(metrics['f1_score']))

accuracy = np.round(np.average(metrics['accuracy']),4)
f1 = np.round(np.average(metrics['f1_score']),4)
              
with open(f'sota/{network}/logs.txt', 'w') as f:
    f.write(f'accuracy: {accuracy}\n')
    f.write(f'f1 score: {f1}\n')
    f.close()

if not multi_class:
    print('sensitivity: ' + str(np.round(np.average(metrics['sensitivity']),4)))
    print('specificity: ' + str(np.round(np.average(metrics['specificity']),4)))

Epoch 1/120
45/45 [==============================] - 36s 732ms/step - loss: 2.0333 - accuracy: 0.4627 - val_loss: 1340.2064 - val_accuracy: 0.0495 - lr: 0.0010
Epoch 2/120
45/45 [==============================] - 33s 741ms/step - loss: 1.4873 - accuracy: 0.6428 - val_loss: 48.4784 - val_accuracy: 0.1094 - lr: 0.0010
Epoch 3/120
45/45 [==============================] - 34s 751ms/step - loss: 1.3649 - accuracy: 0.6983 - val_loss: 11.7274 - val_accuracy: 0.1849 - lr: 0.0010
Epoch 4/120
45/45 [==============================] - 33s 745ms/step - loss: 1.2553 - accuracy: 0.7458 - val_loss: 5.9837 - val_accuracy: 0.1354 - lr: 0.0010
Epoch 5/120
45/45 [==============================] - 34s 746ms/step - loss: 1.1821 - accuracy: 0.7741 - val_loss: 5.0736 - val_accuracy: 0.1302 - lr: 0.0010
Epoch 6/120
45/45 [==============================] - 34s 747ms/step - loss: 1.1264 - accuracy: 0.7964 - val_loss: 2.6160 - val_accuracy: 0.4036 - lr: 0.0010
Epoch 7/120
45/45 [==============================] - 

## Train and save model as TF Lite

In [ ]:
output_directory = 'tflite/'
input_shape = X_train.shape[1:]
nb_classes = len(y_val_oh[0])
verbose = True
nb_epochs = 100 #TBD

np.random.seed(0)
tf.random.set_seed(0)

clf = Classifier_RESNET(output_directory=output_directory, input_shape=input_shape, nb_classes=nb_classes,\
                       verbose=verbose, n_feature_maps=256, batch_size = 256)

clf.fit(X_train, y_train_oh, X_val, y_val_oh, nb_epochs)

### Save model
output_directory = 'models/' #TBD

from keras.models import load_model
model = load_model(output_directory + "best_model.hdf5")

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open(output_directory + "model.tflite", 'wb') as f:
  f.write(tflite_model)